In [24]:
# Standard libraries
import os
import time
import datetime as dt
from functools import wraps
from contextlib import contextmanager
from importlib import reload

# Third-party libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as GridSpec
import seaborn as sns
from scipy import stats

# From project
import portfolio
from portfolio import Portfolio

# API
import refinitiv.data as rd
from refinitiv.data.content import symbol_conversion

# Open session
try:
    rd.open_session(app_key=os.getenv("refinitiv_key")) 
except Exception as e:
    print(f'Unable to open LSEG session')

In [3]:
def retry(n, wait=3):
    """
    Decorator generator to retry execution up to `n` times.

    Parameters
    ----------
    n : int
        Maximum number of attempts.
    wait : int, optional
        Seconds to wait between attempts (default=3).

    Returns
    -------
    function
        A decorator that retries the wrapped function call.
    """
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(1, n + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt < n:
                        print(f'Attempt {attempt}/{n} failed. Retrying in {wait} seconds...')
                        time.sleep(wait)
                    else:
                        print(f'Attempt {attempt}/{n} failed. Max attempts reached.')
                        raise
        return wrapper
    return decorator


# Wrap rd.get_data function to LSEG
@retry(3)
def LSEG(*args, **kwargs):
    """
    Wrapper for rd.get_data with retry logic.
    """
    print('Attempting LSEG retrieval...')
    return rd.get_data(*args, **kwargs)

# Wrap rd.get_history function to LSEG_hist
@retry(3)
def LSEG_hist(*args, **kwargs):
    """
    Wrapper for rd.get_history with retry logic.
    """
    print('Attempting LSEG retrieval...')
    return rd.get_history(*args, **kwargs)


In [ ]:
reload(portfolio)

@contextmanager
def statement_dir():
    """Access directory that stores eToro statements"""
    current_dir = os.getcwd()
    os.chdir(r'C:\Users\cedri\OneDrive\Portfolio Management\eToro Statements')
    try:
        yield
    finally:
        os.chdir(current_dir)

with statement_dir():
    acc_activity = pd.read_excel('cedric_statement.xlsx', sheet_name='Account Activity')

my_pf = Portfolio.from_etoro(acc_activity)

my_pf._holds

In [ ]:
AVGO = rd.get_history(universe=['AVGO.O'], 
                 fields=['TR.CLOSEPRICE(Adjusted=0)'],
                 parameters = {
                    'SDate': '2024-07-01',
                    'EDate': '2024-07-31',
                    'Curn': 'USD',
                    'Frq': 'D',
                    },
                 )
AVGO